to do:
- image in tensorboard (test alpha mobilenet)
- imports aufräumen
- Secondstage.py
- Konfiguration außerhalb:
    - Name output
    - cat/ bin/ reg
- endliche Anzahl an Runden

In [1]:
%tensorflow_version 1.x
#!pip install tensorflow-gpu==1.15
!pip install object-detection-core
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
from google.colab import drive, files
drive.mount('/content/gdrive')

     |████████████████████████████████| 993kB 5.0MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:

from tqdm import tqdm
import re
from datetime import datetime
from multiprocessing import Process, Queue

from copy import deepcopy

from keras import applications, optimizers, backend as K
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Reshape, Conv2D, Activation
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, Callback, LambdaCallback
from keras.utils import np_utils
import pandas as pd
import numpy as np
from numpy.random import randint
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import io
import os
import sys
import csv, json, pickle
from lxml import etree


os.chdir('/content/gdrive/My Drive/')
from SecondStage_colab import second_stage_utils
from SecondStage_colab import file_utils

sys.path.append('/content/gdrive/My Drive/SecondStage_colab/')
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/SecondStage_colab/'

Using TensorFlow backend.


In [0]:
LOG_PATH = '/content/gdrive/My Drive/SecondStage_colab/output/'
LABEL_MAP_PATH = '/content/gdrive/My Drive/SecondStage_colab/label_map.pbtxt'
EVAL_RECORD = '/content/gdrive/My Drive/data/generated/SecondStage_X/validation_rot9_13colors.record'

GPU = True

In [21]:
from file_utils import save_json
from second_stage_utils import *

# you can not set these here, change them in the second_stage_utils
print(ANGLES_PER_BIN)
print(NUM_ORI_BINS)

def train(train_record, conf, out, rep=1):
    timestamp = "{:%Y-%m-%d-%H-%M}".format(datetime.now())
    log_path = LOG_PATH + conf['name'] + '_' + out + '/' + timestamp + '-r' + str(rep) + '/'
    os.makedirs(log_path, exist_ok=True)
    save_json(log_path + '/experiment_config.json', conf)

    label_map = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)

    num_classes = label_map_util.get_max_label_map_index(label_map_util.load_labelmap(LABEL_MAP_PATH)) + 1

    X,Y,Z,_ = tf_record_load_crops([train_record])

    X_train, Y_train, Z_train = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_train = angle_to_bin(Z_train)
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")
    print(Y_train.shape)
    print(num_classes)
    print(Z2_train.shape)
    print(NUM_ORI_BINS)
    print("´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´")

    eval_records = EVAL_RECORD

    X,Y,Z,_ = tf_record_extract_crops([eval_records], 1, 0.0, 0.0)
    X_val, Y_val, Z_val = data_to_keras(X,Y,Z,num_classes,conf['img_size'])
    Z2_val = angle_to_bin(Z_val)

    assert len(X_val) > 0 and len(Y_val) > 0 and len(Z_val) > 0, '{} is incomplete'.format(eval_records)

    outputs = None
    model_final = None
    summary = TensorBoardCustom(log_dir=log_path, label_map=label_map, AddCustomMetrics=(out == '_cat'))
    earlyStop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

    filepath=log_path+"model-{epoch:02d}.h5"
    checkpoint = ModelCheckpoint(filepath, verbose=1, period=20)
    if conf['optimizer'] == 'rmsprop':
        optimizer = optimizers.RMSprop(lr=conf['learning_rate'])
    elif conf['optimizer'] == 'adam':
        optimizer = optimizers.Adam(lr=conf['learning_rate'])
    else:
        raise Error('Unknown optimizer: ' + conf['optimizer'])

    loss_weights = [1.0 if out == '_cat' else 0.0, 1.0 if out == '_reg' else 0.0, 1.0 if out == '_bin' else 0.0]
    epochs = 0
    if out == '_cat': 
      epochs = conf['epochs_cat']
    elif out == '_reg': 
      epochs = conf['epochs_reg']
    else:
      epochs = conf['epochs_bin']

    mobilenet_base = applications.mobilenet.MobileNet(alpha = conf['alpha'],
                                                      weights = "imagenet",
                                                      include_top=False,
                                                      dropout = conf['dropout'],
                                                      input_shape = (
                                                      conf['img_size'],
                                                      conf['img_size'],
                                                        3
                                                      ))
    shape = (1, 1, int(1024 * conf['alpha']))
    x = GlobalAveragePooling2D()(mobilenet_base.output)
    x = Reshape(shape, name='reshape_1')(x)
    #x = Dropout(conf['dropout'], name='dropout')(x)
    # Branch regression
    reg = Conv2D(1, (1, 1), padding='same', name='conv_reg')(x)
    reg = Activation('linear', name='act_linear')(reg)
    reg = Reshape((1,), name='reg_out')(reg)
    # Branch orientation classification with bins
    bin = Conv2D(NUM_ORI_BINS, (1, 1), padding='same', name='conv_bin')(x)
    bin = Activation('softmax', name='act_bin')(bin)
    bin = Reshape((NUM_ORI_BINS,), name='bin_out')(bin)
    # Branch classification
    cat = Conv2D(num_classes, (1, 1), padding='same', name='conv_cat')(x)
    cat = Activation('softmax', name='act_softmax')(cat)
    cat = Reshape((num_classes,), name='cat_out')(cat)

    model_final = Model(inputs = mobilenet_base.input, outputs = [cat,reg,bin])
    model_final.compile(optimizer = optimizer,
                  loss={'cat_out': 'categorical_crossentropy',
                        'reg_out': angle_mse,
                        'bin_out': 'categorical_crossentropy',
                  },
                  loss_weights={'cat_out': loss_weights[0],
                                'reg_out': loss_weights[1],
                                'bin_out': loss_weights[2]},
                  metrics ={'cat_out': 'accuracy',
                            'reg_out': angle_mae,
                            'bin_out': angle_bin_rmse})
    model_final.fit(
        X_train,
        {'cat_out': Y_train, 'reg_out': Z_train, 'bin_out': Z2_train},
        validation_data=(X_val, [Y_val, Z_val, Z2_val]),
        batch_size=conf['batch_size'], epochs=epochs, verbose=1,
        callbacks=[summary, checkpoint, earlyStop],
        shuffle=True
    )

    model_final.save(log_path+"model-final.h5")
    print(log_path)
    print("Finished training for {}".format(conf['name']))

1
360


In [0]:
default_sstage_conf = {
    'dataset': 'default',
    'epochs_cat': 70,
    'epochs_reg': 60,
    'epochs_bin': 70,
    'optimizer': 'adam',
    'learning_rate': 1e-4,
    'dropout': 0.001,
    'alpha': 0.5,
    'img_size': 35,
    'repetions': 1,
    'batch_size': 4096,
}

def create_all_sstage_experiments():
    configs = []
    #configs.extend(create_sstage_default())
    configs.extend(create_sstage_dropouts_alphas())
    return configs

def create_sstage_dropouts_alphas():
    config = []
    dropouts = [0.001, 0.1, 0.2, 0.3]
    alphas = [0.5, 0.75]
    for drop in dropouts:
      modified = deepcopy(default_sstage_conf)
      modified['name'] = "ssdef_drop" + str(drop)
      modified['dropout'] = drop
      for alp in alphas:
        modifiedB = deepcopy(modified)
        modifiedB['name'] = modifiedB['name'] + "_alpha" + str(alp)
        modifiedB['alpha'] = alp
        config.append(deepcopy(modifiedB))
    return config

def create_sstage_default():
    config = []
    modified = deepcopy(default_sstage_conf)
    modified['name'] = "sstage_default"
    config.append(deepcopy(modified))
    return config


exp = create_all_sstage_experiments()
TORUN = []
TORUN.append('_bin')
TORUN.append('_cat')
#TORUN.append('_reg')
train_records = ['training_rot12_13colors.record']#, 'training_rot9_13colors.record']
for config in tqdm(exp):
    print("or_conf", config)
    for r in range(config['repetions']):     
        for out in TORUN: # 
            for train_record in train_records:
              conf = deepcopy(config)
              conf['name'] = conf['name']+'_'+out+"_"+train_record
              print("Start training:  " + conf['name'])
              
              if not GPU:
                  p = Process(
                    target=train,
                    args=("/content/gdrive/My Drive/data/generated/SecondStage_X/" + train_record, conf, out, r)
                  )
                  p.start() 
              else:
                train("/content/gdrive/My Drive/data/generated/SecondStage_X/" + train_record, conf, out)









  0%|          | 0/8 [00:00<?, ?it/s]

or_conf {'dataset': 'default', 'epochs_cat': 70, 'epochs_reg': 60, 'epochs_bin': 70, 'optimizer': 'adam', 'learning_rate': 0.0001, 'dropout': 0.001, 'alpha': 0.5, 'img_size': 35, 'repetions': 1, 'batch_size': 4096, 'name': 'ssdef_drop0.001_alpha0.5'}
Start training:  ssdef_drop0.001_alpha0.5__bin_training_rot12_13colors.record
Loaded 280800 crops from ['/content/gdrive/My Drive/data/generated/SecondStage_X/training_rot12_13colors.record']
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´
(280800, 14)
14
(280800, 360)
360
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Train on 280800 samples, validate on 81000 samples
Epoch 1/70
280800/280800 [==============================] - 26s 92us/step - loss: 6.8955 - cat_out_loss: 4.0830 - reg_out_loss: 10790.0582 - bin_out_loss: 6.8955 - cat_out_acc: 0.0779 - reg_out_angle_mae: 89.9681 - bin_out_angle_bin_rmse: 103.4239 - val_loss: 7.5092 - val_cat_out_loss: 5.6885 - val_reg_out_loss: 10792.2386 - val_bin_out_loss: 7.5092 - val_cat_out_acc: 0.0795 - val_reg_out_angle_mae: 90.0512 - val_bin_out_angle_bin_rmse: 103.3192
Epoch 2/70
280800/280800 [==============================] - 18s 65us/step - loss: 5.9033 - cat_out_loss: 3.1932 - reg_out_loss: 10793.2349 - bin_out_loss: 5.9033 - cat_out_acc: 0.0708 - reg_out_angle_mae: 89.9856 - bin_out_angle_bin_rmse: 99.1695 - val_loss: 6.2508 - val_cat_out_loss: 4.6119 - val_reg_out_loss: 10781.6742 - val_bin_out_loss: 6.2508 - val_cat_out_acc: 0.0742 - val_reg_out_angle_mae: 89.9922 - val_bin_out_angle_bin_rmse: 94.9882
Epoch 3/70
280800/280800 [=========================

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


...

Epoch 2/70
280800/280800 [==============================] - 18s 65us/step - loss: 1.2887 - cat_out_loss: 1.2887 - reg_out_loss: 10784.0069 - bin_out_loss: 7.3051 - cat_out_acc: 0.5557 - reg_out_angle_mae: 89.9711 - bin_out_angle_bin_rmse: 104.1399 - val_loss: 1.2440 - val_cat_out_loss: 1.2440 - val_reg_out_loss: 10787.8279 - val_bin_out_loss: 8.6250 - val_cat_out_acc: 0.6402 - val_reg_out_angle_mae: 89.9623 - val_bin_out_angle_bin_rmse: 104.0499
Epoch 3/70
280800/280800 [==============================] - 18s 65us/step - loss: 0.6332 - cat_out_loss: 0.6332 - reg_out_loss: 10788.6251 - bin_out_loss: 7.3574 - cat_out_acc: 0.7700 - reg_out_angle_mae: 89.9856 - bin_out_angle_bin_rmse: 104.3927 - val_loss: 0.7725 - val_cat_out_loss: 0.7725 - val_reg_out_loss: 10787.7365 - val_bin_out_loss: 8.2843 - val_cat_out_acc: 0.7623 - val_reg_out_angle_mae: 89.9764 - val_bin_out_angle_bin_rmse: 104.4769
Epoch 4/70
280800/280800 [==============================] - 18s 65us/step - loss: 0.3544 - cat_out_l









 12%|█▎        | 1/8 [1:11:15<8:18:49, 4275.67s/it]

or_conf {'dataset': 'default', 'epochs_cat': 70, 'epochs_reg': 60, 'epochs_bin': 70, 'optimizer': 'adam', 'learning_rate': 0.0001, 'dropout': 0.001, 'alpha': 0.75, 'img_size': 35, 'repetions': 1, 'batch_size': 4096, 'name': 'ssdef_drop0.001_alpha0.75'}
Start training:  ssdef_drop0.001_alpha0.75__bin_training_rot12_13colors.record
Loaded 280800 crops from ['/content/gdrive/My Drive/data/generated/SecondStage_X/training_rot12_13colors.record']
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´
(280800, 14)
14
(280800, 360)
360
´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´´


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Train on 280800 samples, validate on 81000 samples
Epoch 1/70
280800/280800 [==============================] - 31s 111us/step - loss: 6.5857 - cat_out_loss: 3.6755 - reg_out_loss: 10800.1427 - bin_out_loss: 6.5857 - cat_out_acc: 0.0657 - reg_out_angle_mae: 90.0029 - bin_out_angle_bin_rmse: 103.8398 - val_loss: 6.6461 - val_cat_out_loss: 4.6915 - val_reg_out_loss: 10809.2239 - val_bin_out_loss: 6.6461 - val_cat_out_acc: 0.0681 - val_reg_out_angle_mae: 89.9879 - val_bin_out_angle_bin_rmse: 103.6316
Epoch 2/70
280800/280800 [==============================] - 23s 82us/step - loss: 5.7542 - cat_out_loss: 2.8592 - reg_out_loss: 10801.5707 - bin_out_loss: 5.7542 - cat_out_acc: 0.0705 - reg_out_angle_mae: 90.0086 - bin_out_angle_bin_rmse: 102.9701 - val_loss: 5.6228 - val_cat_out_loss: 3.5136 - val_reg_out_loss: 10803.2477 - val_bin_out_loss: 5.6228 - val_cat_out_acc: 0.0659 - val_reg_out_angle_mae: 90.0353 - val_bin_out_angle_bin_rmse: 99.0264
Epoch 3/70
280800/280800 [=======================

In [0]:
%load_ext tensorboard
%reload_ext tensorboard
# LOG_PATH did not work for some reason
%tensorboard --logdir '/content/gdrive/My Drive/SecondStage_colab/output/'

In [0]:
!pip install -U tensorboard >piplog 2>&1
!tensorboard dev upload --logdir '/content/gdrive/My Drive/SecondStage_colab/output/'

In [0]:
!pip install -U tensorboard >piplog 2>&1
!tensorboard dev list
# You must replace YOUR_EXPERIMENT_ID with the value output from the previous
# tensorboard `list` command or `upload` command.  For example
# `tensorboard dev delete --experiment_id pQpJNh00RG2Lf1zOe9BrQA`

## !tensorboard dev delete --experiment_id YOUR_EXPERIMENT_ID_HERE

In [0]:
!rm "/content/gdrive/My Drive/SecondStage_colab/output/" -rf
!ls "/content/gdrive/My Drive/SecondStage_colab/"